<a href="https://colab.research.google.com/github/lsteffenel/NumbaCuda/blob/main/Effective_Memory_Use.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Avant de commencer
L'exécution de ces notebooks sur Colab nécessite deux choses (au 4/2/2025) :

1. des resources GPU
  * Menu "Exécution" -> "Modifier le type d'exécution"
2. D'utiliser une version plus ancienne de Colab en raison de certaines incompatibilités du pilote Nvidia
  * Connecter l'environnement d'exécution
  * Menu "Outils" -> "Pallette de commandes". Cherchez "version" dans la barre et sélectionnez l'option "Utiliser la version d'environnement d'exécution de remplacement"


# Utilisation efficace du sous-système de mémoire

Maintenant que vous savez écrire des noyaux CUDA  et que vous comprenez l'importance de lancer des grilles pour donner suffisamment de travail au GPU afin de masquer la latence, vous allez apprendre des techniques pour utiliser efficacement la mémoire du GPU. Ces techniques sont largement applicables à une variété d'applications CUDA et sont parmi les plus importantes lorsqu'il s'agit d'accélérer votre code CUDA.

Vous allez commencer par en apprendre davantage sur la coalescence de mémoire (regroupement/organisation de blocs mémoire). Pour tester votre capacité à raisonner sur la coalescence, vous découvrirez ensuite les grilles bidimensionnelles et les blocs de threads. Ensuite, vous découvrirez comment utiliser la mémoire partagée, qui sera utilisée pour faciliter la coalescence là où cela n'aurait pas été possible autrement. Enfin, vous découvrirez les conflits qui peuvent arriver avec la mémoire partagée et une technique pour les résoudre.


## Le problème : l'accès "éparpillé" à la mémoire nuit les performances

Avant d’apprendre les détails sur la coalescence, exécutez les cellules suivantes pour observer les implications en termes de performances d’un changement apparemment trivial du mode d’accès aux données.

### Imports

In [ ]:
import numpy as np
from numba import cuda

### Data Creation

Dans cette cellule, nous définissons `n` et créons une grille avec `n` threads. Nous créons également un vecteur de sortie de longueur `n`. Pour les entrées, nous créons des vecteurs de taille `stride * n` pour des raisons qui seront expliquées ci-dessous :

In [ ]:
n = 1024*1024 # 1M

threads_per_block = 1024
blocks = int(n / threads_per_block)

stride = 16

# Input Vectors of length stride * n
a = np.ones(stride * n).astype(np.float32)
b = a.copy().astype(np.float32)

# Output Vector
out = np.zeros(n).astype(np.float32)

d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_out = cuda.to_device(out)

### Kernel Definition

Dans `add_experiment`, chaque thread de la grille ajoutera un élément dans `a` et un élément dans `b` puis écrira le résultat dans `out`. Le noyau a été écrit de telle sorte que nous puissions passer une valeur `coalesced` de `True` ou `False` pour affecter la façon dont il indexe dans les vecteurs `a` et `b`. Vous verrez la comparaison des performances des deux modes ci-dessous.

In [ ]:
@cuda.jit
def add_experiment(a, b, out, stride, coalesced):
    i = cuda.grid(1)
    # The above line is equivalent to
    # i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    if coalesced == True:
        out[i] = a[i] + b[i]
    else:
        out[i] = a[stride*i] + b[stride*i]

### Lancement d'un kernet avec un accès "coalesced"

Ici, nous passons « True » comme valeur « coalesced » et observons les performances du noyau sur plusieurs exécutions :

In [ ]:
%timeit add_experiment[blocks, threads_per_block](d_a, d_b, d_out, stride, True); cuda.synchronize

Vérifions si le noyau s'exécute comme attendu :

In [ ]:
result = d_out.copy_to_host()
truth = a[:n] + b[:n]

In [ ]:
np.array_equal(result, truth)

### Lancement d'un noyau sans accès coalescent

Dans cette cellule, nous passons " False " pour observer les performances du modèle d'accès aux données non coalescents :

In [ ]:
%timeit add_experiment[blocks, threads_per_block](d_a, d_b, d_out, stride, False); cuda.synchronize

Vérifions si le noyau s'exécute comme attendu :

In [ ]:
result = d_out.copy_to_host()
truth = a[::stride] + b[::stride]

In [ ]:
np.array_equal(result, truth)

### Résultats

Les performances du mode d'accès "non coalescent" sont bien pires. Vous allez maintenant découvrir pourquoi et comment réfléchir aux modes d'accès aux données pour obtenir des noyaux très performants.

## Présentation : Global Memory Coalescing

Regardez la présentation ci-dessous :

In [ ]:
from IPython.display import IFrame
IFrame('https://view.officeapps.live.com/op/view.aspx?src=https://developer.download.nvidia.com/training/courses/C-AC-02-V1/coalescing-v3.pptx', 800, 450)

## Exercice : Somme des Colonnes et Lignes

Pour cet exercice, il vous sera demandé d'écrire un noyau pour faire la somme des colonnes, utilisant le mode d'accès mémoire coalescés. Pour commencer, vous observerez les performances sans ce mode d' accès mémoire.

### Somme des lignes

**Imports**

In [ ]:
import numpy as np
from numba import cuda

**Data Creation**

Dans ce paragraphe nous créons une matrice pour l'entrée ainsi qu'un vecteur pour stocker la solution, et nous transférons chacun d'eux vers le périphérique. Nous définissons également les dimensions de la grille et du bloc à utiliser lorsque nous lançons le noyau.


In [ ]:
n = 16384 # matrix side size
threads_per_block = 256
blocks = int(n / threads_per_block)

# Input Matrix
a = np.ones(n*n).reshape(n, n).astype(np.float32)
# Here we set an arbitrary row to an arbitrary value to facilitate a check for correctness below.
a[3] = 9

# Output vector
sums = np.zeros(n).astype(np.float32)

d_a = cuda.to_device(a)
d_sums = cuda.to_device(sums)

**Le noyau**

`row_sums` utilisera chaque thread pour parcourir une ligne de données, effectuer la somme, puis stockera la somme des lignes dans `sums`.

In [ ]:
@cuda.jit
def row_sums(a, sums, n):
    idx = cuda.grid(1)
    sum = 0.0

    for i in range(n):
        # Each thread will sum a row of `a`
        sum += a[idx][i]

    sums[idx] = sum

**Performance**

In [ ]:
%timeit row_sums[blocks, threads_per_block](d_a, d_sums, n); cuda.synchronize()

**Vérification du résultat**

In [ ]:
result = d_sums.copy_to_host()
truth = a.sum(axis=1)

In [ ]:
np.array_equal(truth, result)

### Somme des colonnes

**Imports**

In [ ]:
import numpy as np
from numba import cuda

**Data Creation**

On reprend le même format précédent, mais avec des valeurs sur les colonnes

In [ ]:
n = 16384 # matrix side size
threads_per_block = 256
blocks = int(n / threads_per_block)

a = np.ones(n*n).reshape(n, n).astype(np.float32)
# Here we set an arbitrary column to an arbitrary value to facilitate a check for correctness below.
a[:, 3] = 9
sums = np.zeros(n).astype(np.float32)

d_a = cuda.to_device(a)
d_sums = cuda.to_device(sums)

**Définition du noyau**

`col_sums` utilisera chaque thread pour parcourir une colonne de données, en la sommant, puis stockera la somme de sa colonne dans `sums`. Complétez la définition du noyau pour y parvenir (c'est à vous de le faire 😀)

In [ ]:
@cuda.jit
def col_sums(a, sums, ds):
    # TODO: Write this kernel to store the sum of each column in matrix `a` to the `sums` vector.
    pass

**Vérification de la Performance**

En supposant que vous ayez écrit `col_sums` pour utiliser l'accès coalescent, vous devriez voir une accélération significative (presque 2x) par rapport aux `row_sums` "non coalescent" que vous avez exécutés ci-dessus :

In [ ]:
%timeit col_sums[blocks, threads_per_block](d_a, d_sums, n); cuda.synchronize()

**Vérification des résultats**

Confirm your kernel is working as expected.

In [ ]:
result = d_sums.copy_to_host()
truth = a.sum(axis=0)

In [ ]:
np.array_equal(truth, result)

## Des blocs et grilles à 2 et 3 dimensions

Les grilles et les blocs peuvent être configurés pour contenir respectivement une collection bidimensionnelle ou tridimensionnelle de blocs ou de threads. Cela est fait principalement pour des raisons de commodité pour les programmeurs qui travaillent avec des données bidimensionnels ou tridimensionnels. Voici un exemple très simple pour mettre en évidence la syntaxe. Il faudra comprendre la définition du noyau et comme il est lancé pour que le concept n'ait un sens.

In [ ]:
import numpy as np
from numba import cuda

In [ ]:
A = np.zeros((4,4)) # A 4x4 Matrix of 0's
d_A = cuda.to_device(A)

# Here we create a 2D grid with 4 blocks in a 2x2 structure, each with 4 threads in a 2x2 structure
# by using a Python tuple to signify grid and block dimensions.
blocks = (2, 2)
threads_per_block = (2, 2)

Ce noyau prendra une matrice d'entrée de 0 et écrira chacun de ses éléments directement dans la grille au format `X.Y`` :

In [ ]:
@cuda.jit
def get_2D_indices(A):
    # By passing `2`, we get the thread's unique x and y coordinates in the 2D grid
    x, y = cuda.grid(2)
    # The above is equivalent to the following 2 lines of code:
    # x = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    # y = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y

    # Write the x index followed by a decimal and the y index.
    A[x][y] = x + y / 10

In [ ]:
get_2D_indices[blocks, threads_per_block](d_A)

In [ ]:
result = d_A.copy_to_host()
result

## Exercice : Somme de matrices 2D en mode coalescent

### Imports

In [ ]:
import numpy as np
from numba import cuda

### Data Creation

Dans cette cellule, nous définissons des matrices d'entrée d'éléments 2048x2048 `a` et `b`, ainsi qu'une matrice de sortie initialisée de 2048x2048. Nous copions ces matrices sur le GPU.

Nous définissons également les dimensions de bloc et de grille à 2 dimensions. Notez que nous créons une grille avec le même nombre total de threads que d'éléments d'entrée et de sortie, de sorte que chaque thread de la grille calculera la somme pour un seul élément de la matrice de sortie.

In [ ]:
n = 2048*2048 # 4M

# 2D blocks
threads_per_block = (32, 32)
# 2D grid
blocks = (64, 64)

# 2048x2048 input matrices
a = np.arange(n).reshape(2048,2048).astype(np.float32)
b = a.copy().astype(np.float32)

# 2048x2048 0-initialized output matrix
out = np.zeros_like(a).astype(np.float32)

d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_out = cuda.to_device(out)

### Somme pour une matrice 2D

Votre tâche consiste à compléter les tâches à effectuer dans `matrix_add` pour additionner correctement `a` et `b` dans `out`. Pour vous aider à comprendre les modes d'accès, `matrix_add` acceptera un booléen `coalesced` indiquant si les modèles d'accès doivent être coalescents ou non. Les deux modes (coalesced et uncoalesced) devraient produire des résultats corrects, cependant, vous devriez observer des accélérations significatives ci-dessous lors de l'exécution avec `coalesced` défini sur `True`.

In [ ]:
@cuda.jit
def matrix_add(a, b, out, coalesced):
    # TODO: set x and y to index correctly such that each thread
    # accesses one element in the data.
    x, y = pass

    if coalesced == True:
        # TODO: write the sum of one element in `a` and `b` to `out`
        # using a coalesced memory access pattern.
    else:
        # TODO: write the sum of one element in `a` and `b` to `out`
        # using an uncoalesced memory access pattern.

### Vérification de la performance

Exécutez les deux cellules ci-dessous pour lancer `matrix_add` avec les modèles d'accès que vous avez écrits, et observez la différence de performances. Des cellules supplémentaires ont été fournies pour confirmer l'exactitude de votre noyau.

**Coalesced**

In [ ]:
%timeit matrix_add[blocks, threads_per_block](d_a, d_b, d_out, True); cuda.synchronize

In [ ]:
result = d_out.copy_to_host()
truth = a+b

In [ ]:
np.array_equal(result, truth)

**Uncoalesced**

In [ ]:
%timeit matrix_add[blocks, threads_per_block](d_a, d_b, d_out, False); cuda.synchronize

In [ ]:
result = d_out.copy_to_host()
truth = a+b

In [ ]:
np.array_equal(result, truth)

## Mémoire Partagée

Jusqu'à présent, nous avons fait la distinction entre la mémoire de l'hôte et celle de la GPU, comme si la mémoire GPU était un seul type de mémoire. Mais en fait, CUDA a une [hiérarchie de mémoire](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#memory-hierarchy) encore plus fine. La mémoire du dispositif que nous avons utilisée jusqu'à présent est appelée **mémoire globale**, disponible pour n'importe quel thread ou bloc sur l'appareil, et qui peut persister pendant toute la durée de vie de l'application. Naturellement, c'est un espace mémoire relativement grand.

Nous allons maintenant discuter de la manière d'utiliser une région de la mémoire appelée **mémoire partagée**. La mémoire partagée est un cache défini par le programmeur, et de taille limitée qui [dépend du GPU](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capabilities). Elle est **partagée** entre tous les threads d'un bloc. Il s'agit d'une ressource rare, à laquelle les threads extérieurs au bloc ne peuvent pas accéder et qui ne persiste pas après la fin de l'exécution d'un noyau. La mémoire partagée a cependant une bande passante beaucoup plus élevée que la mémoire globale et peut être utilisée avec plus d'efficacité dans de nombreux noyaux, en particulier pour optimiser les performances.

Voici quelques cas d'utilisation courants de la mémoire partagée :

* Mise en cache de la mémoire lue à partir de la mémoire globale qui devra être lue plusieurs fois dans un bloc.
* Mise en mémoire tampon de la sortie des threads afin qu'elle puisse être fusionnée (coalescence) avant de la réécrire dans la mémoire globale.
* Stockage temporaire de données utilisés dans des opérations gather/scatter dans un bloc.


### La syntaxe ppour la mémoire partagée

Numba fournit des [fonctions](https://numba.pydata.org/numba-doc/dev/cuda/memory.html#shared-memory-and-thread-synchronization) pour l'allocation de mémoire partagée ainsi que pour la synchronisation entre les threads d'un bloc, ce qui est souvent nécessaire après que des threads parallèles ont lu ou écrit dans la mémoire partagée.

Lorsque vous déclarez une mémoire partagée, vous fournissez la forme du tableau partagé, ainsi que son type, à l'aide d'un [type Numba](https://numba.pydata.org/numba-doc/dev/reference/types.html#numba-types). **La forme du tableau doit être une valeur constante**, et par conséquent, vous ne pouvez pas utiliser d'arguments passés à la fonction ni des variables fournies comme `numba.cuda.blockDim.x`, ou les valeurs calculées de `cuda.griddim`.

Comme ce concepte est un peu plus compliqué, voici un exemple pour démontrer la syntaxe avec des commentaires soulignant le mouvement de la mémoire hôte vers la mémoire globale du dispositif, vers la mémoire partagée, vers la mémoire globale du périphérique et enfin vers la mémoire hôte :


**Imports**

Nous utiliserons `numba.types` pour définir les types de valeurs dans la mémoire partagée.

In [ ]:
import numpy as np
from numba import types, cuda

**<Echange (swap) d'éléments avec la mémoire partagée**

Le noyau suivant prend un vecteur d'entrée, où chaque thread écrira d'abord un élément dans la mémoire partagée puis, après avoir synchronisé de telle sorte que tous les éléments aient été écrits dans la mémoire partagée, écrira un élément de la mémoire partagée dans le vecteur de sortie.

Il convient de noter que chaque thread écrira une valeur à partir d'une position de la mémoire partagée, laquelle a été écrite par un autre thread.

In [ ]:
@cuda.jit
def swap_with_shared(vector, swapped):
    # Allocate a 4 element vector containing int32 values in shared memory.
    temp = cuda.shared.array(4, dtype=types.int32)

    idx = cuda.grid(1)

    # Move an element from global memory into shared memory
    temp[idx] = vector[idx]

    # cuda.syncthreads will force all threads in the block to synchronize here, which is necessary because...
    cuda.syncthreads()
    #...the following operation is reading an element written to shared memory by another thread.

    # Move an element from shared memory back into global memory
    swapped[idx] = temp[3 - cuda.threadIdx.x] # swap elements

**Data Creation**

In [ ]:
vector = np.arange(4).astype(np.int32)
swapped = np.zeros_like(vector)

# Move host memory to device (global) memory
d_vector = cuda.to_device(vector)
d_swapped = cuda.to_device(swapped)

In [ ]:
vector

**Exécution du Kernel**

In [ ]:
swap_with_shared[1, 4](d_vector, d_swapped)

**Vérifier les Résultats**

In [ ]:
# Move device (global) memory back to the host
result = d_swapped.copy_to_host()
result

## Présentation : Mémoire partagée en mode coalescence

Exécutez la cellule suivante pour charger les diapositives, puis cliquez sur « Démarrer le diaporama » pour les mettre en plein écran.

In [ ]:
from IPython.display import IFrame
IFrame('https://view.officeapps.live.com/op/view.aspx?src=https://developer.download.nvidia.com/training/courses/C-AC-02-V1/shared_coalescing.pptx', 800, 450)

## Exercice : Utilisation de la mémoire partagée pour les lectures et écritures fusionnées avec transposition de matrice

Dans cet exercice, vous allez mettre en œuvre ce qui vient d'être démontré dans la présentation en écrivant un noyau pour la transposition d'une matrice qui, en utilisant la mémoire partagée, effectue des lectures et des écritures coalescées dans la matrice de sortie localisée dans la mémoire globale.

### Lectures coalescentes, écritures non-coalescentes

À titre de comparaison des performances, voici un noyau de transposition de matrice naïf qui effectue des lectures coalescées à partir de l'entrée, mais des écritures non coalescées vers la sortie.

**Imports**

In [ ]:
from numba import cuda
import numpy as np

**Data Creation**

Ici, nous créons une matrice d'entrée 4096x4096 `a` ainsi qu'une matrice de sortie 4096x4096 `transposée`, et les copions sur l'appareil.

Nous définissons également une grille bidimensionnelle avec des blocs bidimensionnels à utiliser ci-dessous. Notez que nous avons créé une grille avec un nombre total de threads égal au nombre d'éléments dans la matrice d'entrée.

In [ ]:
n = 4096*4096 # 16M

# 2D blocks
threads_per_block = (32, 32)
#2D grid
blocks = (128, 128)

# 4096x4096 input and output matrices
a = np.arange(n).reshape((4096,4096)).astype(np.float32)
transposed = np.zeros_like(a).astype(np.float32)

d_a = cuda.to_device(a)
d_transposed = cuda.to_device(transposed)

**Naive Matrix Transpose Kernel**

Ce noyau transpose correctement « a », en écrivant la transposition dans « transposed ». Il effectue des lectures depuis « a » de manière coalescée, cependant, ses écritures dans « transposed » ne sont pas coalescées.

In [ ]:
@cuda.jit
def transpose(a, transposed):
    x, y = cuda.grid(2)

    transposed[x][y] = a[y][x]

**Check Performance**

In [ ]:
%timeit transpose[blocks, threads_per_block](d_a, d_transposed); cuda.synchronize()

**Check Correctness**

In [ ]:
result = d_transposed.copy_to_host()
expected = a.T

In [ ]:
np.array_equal(result, expected)

### Exercice : réécrire le code pour que les lectures et écritures soient coalescentes

Votre travail consistera à refactoriser le noyau « transpose » pour utiliser la mémoire partagée et effectuer à la fois des lectures et des écritures depuis la mémoire globale de manière fusionnée.

**Imports**

In [ ]:
import numpy as np
from numba import cuda, types as numba_types

**Data Creation**

In [ ]:
n = 4096*4096 # 16M

# 2D blocks
threads_per_block = (32, 32)
#2D grid
blocks = (128, 128)

# 4096x4096 input and output matrices
a = np.arange(n).reshape((4096,4096)).astype(np.float32)
transposed = np.zeros_like(a).astype(np.float32)

d_a = cuda.to_device(a)
d_transposed = cuda.to_device(transposed)

**Écrire un noyau de transposition qui utilise la mémoire partagée**

Complétez les `TODO` dans la définition du noyau `tile_transpose`.

In [ ]:
@cuda.jit
def tile_transpose(a, transposed):
    # `tile_transpose` assumes it is launched with a 32x32 block dimension,
    # and that `a` is a multiple of these dimensions.

    # 1) Create 32x32 shared memory array.

    # TODO: Your code here.

    # Compute offsets into global input array. Recall for coalesced access we want to map threadIdx.x increments to
    # the fastest changing index in the data, i.e. the column in our array.
    # Note: `a_col` and `a_row` are already correct.
    a_col = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    a_row = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y

    # 2) Make coalesced read from global memory (using grid indices)
    # into shared memory array (using thread indices).

    # TODO: Your code here.

    # 3) Wait for all threads in the block to finish updating shared memory.

    # TODO: Your code here.

    # 4) Calculate transposed location for the shared memory array tile
    # to be written back to global memory. Note that blockIdx.y*blockDim.y
    # and blockIdx.x* blockDim.x are swapped (because we want to write to the
    # transpose locations), but we want to keep access coalesced, so match up the
    # threadIdx.x to the fastest changing index, i.e. the column./
    # Note: `t_col` and `t_row` are already correct.
    t_col = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.x
    t_row = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.y

    # 5) Write from shared memory (using thread indices)
    # back to global memory (using grid indices)
    # transposing each element within the shared memory array.

    # TODO: Your code here.

**Check Performance**

Vérifiez les performances de votre noyau de transposition refactorisé. Vous devriez constater une accélération par rapport aux performances de transposition de base ci-dessus.

In [ ]:
%timeit tile_transpose[blocks, threads_per_block](d_a, d_transposed); cuda.synchronize()

**Check Correctness**

In [ ]:
result = d_transposed.copy_to_host()
expected = a.T

In [ ]:
np.array_equal(result, expected)

### Pourquoi si peu de gains ?

Bien qu'il s'agisse d'une accélération significative pour seulement quelques lignes de code, vous pourriez penser que l'amélioration des performances n'est pas aussi marquée que vous l'espériez par rapport aux expériences précédentes. Il y a deux raisons principales à cela :

1. Le noyau de transposition naïf effectuait des lectures fusionnées, donc votre version refactorisée n'a optimisé que la moitié de l'accès à la mémoire globale tout au long de l'exécution du noyau.
2. Votre code tel qu'il est écrit souffre de conflits de mémoire partagée, un sujet sur lequel nous allons maintenant porter notre attention.


## Présentation : Conflits de mémoire

Exécutez la cellule suivante pour charger les diapositives, puis cliquez sur « Démarrer le diaporama » pour les mettre en plein écran.

In [ ]:
from IPython.display import IFrame
IFrame('https://view.officeapps.live.com/op/view.aspx?src=https://developer.download.nvidia.com/training/courses/C-AC-02-V1/bank_conflicts.pptx', 800, 450)

## Exercice : Résourdre les conflits de mémoire

En guise d'exercice final, vous refactoriserez le noyau de transposition en utilisant la mémoire partagée pour qu'il soit exempt de conflit de banque de mémoire partagée.

### Imports

In [ ]:
import numpy as np
from numba import cuda, types as numba_types

### Data Creation

In [ ]:
n = 4096*4096 # 16M
threads_per_block = (32, 32)
blocks = (128, 128)

a = np.arange(n).reshape((4096,4096)).astype(np.float32)
transposed = np.zeros_like(a).astype(np.float32)

d_a = cuda.to_device(a)
d_transposed = cuda.to_device(transposed)

### Rendre le noyau sans conflit

Le noyau `tile_transpose_conflict_free` est un noyau de transposition de matrice qui utilise la mémoire partagée de sorte que les lectures et les écritures dans la mémoire globale soient fusionnées. Votre travail consiste à refactoriser le noyau afin qu'il ne souffre plus ces conflits.

In [ ]:
@cuda.jit
def tile_transpose_conflict_free(a, transposed):
    # `tile_transpose` assumes it is launched with a 32x32 block dimension,
    # and that `a` is a multiple of these dimensions.

    # 1) Create 32x32 shared memory array.
    tile = cuda.shared.array((32, 32), numba_types.int32)

    # Compute offsets into global input array.
    x = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    y = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y

    # 2) Make coalesced read from global memory into shared memory array.
    # Note the use of local thread indices for the shared memory write,
    # and global offsets for global memory read.
    tile[cuda.threadIdx.y, cuda.threadIdx.x] = a[y, x]

    # 3) Wait for all threads in the block to finish updating shared memory.
    cuda.syncthreads()

    # 4) Calculate transposed location for the shared memory array tile
    # to be written back to global memory.
    t_x = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.x
    t_y = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.y

    # 5) Write back to global memory,
    # transposing each element within the shared memory array.
    transposed[t_y, t_x] = tile[cuda.threadIdx.x, cuda.threadIdx.y]

### Check Performance

En supposant que vous ayez correctement résolu les conflits, ce noyau devrait s'exécuter beaucoup plus rapidement que le noyau de transposition naïf et que le noyau de transposition à mémoire partagée (avec conflits). Pour réussir l'évaluation, votre noyau devra s'exécuter en moyenne en moins de 840 µs.

La première valeur imprimée en exécutant la cellule suivante vous donnera le temps d'exécution moyen de votre noyau.

In [ ]:
%timeit tile_transpose_conflict_free[blocks, threads_per_block](d_a, d_transposed); cuda.synchronize()